# Init & Login

In [1]:
from math import ceil
from random import randrange
from time import sleep

import spotipy
import spotipy.util as util
from IPython.display import clear_output

## Client Info ##
CLIENT_ID     = ""
CLIENT_SECRET = ""
CLIENT_SCOPE  = "user-follow-modify playlist-modify-private playlist-modify-public"
USER_NAME     = "31ytgsr7wdmiaroy77msqpiupdsi"
REDIR_URI     = "https://github.com/jwatson-CO-edu/yt_shuffle_so_good"
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'
## API Info ##
_RESPONSE_LIMIT = 100

with open( "../keys/spot_ID.txt" , 'r' ) as f:
    CLIENT_ID = f.readlines()[0].strip()

with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
    CLIENT_SECRET = f.readlines()[0].strip()

token = None
token = util.prompt_for_user_token(
    username      = USER_NAME,
    scope         = CLIENT_SCOPE,
    client_id     = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    redirect_uri  = REDIR_URI
)

print( token )

spot = spotipy.Spotify( auth = token )
clear_output( wait = True )
sleep( 2 )
print( "TOKEN OBTAINED" )

TOKEN OBTAINED


# Playlist Functions

In [2]:

def get_playlist_length( playlist_ID ):
    """ Get the number of total tracks in the playlist """
    response = spot.user_playlist_tracks(
        CLIENT_ID, 
        playlist_ID, 
        fields = 'items,uri,name,id,total', 
        limit  = _RESPONSE_LIMIT
    )
    return response['total']


def reorder_playlist_by_chunks( playlist_ID, chunkSize, Nmoves = None, verbose = False ):
    """ Shuffle the playlist a chunk at a time """
    Ntracks = get_playlist_length( playlist_ID )
    Nchunks = int( ceil( Ntracks / chunkSize ) )
    if Nmoves is None:
        Nmoves = int( Ntracks / chunkSize )
    for i in range( Nmoves ):
        fail = True
        while fail:
            bgnDex   = randrange(0,Nchunks) * chunkSize
            moveSize = min( chunkSize, Ntracks - bgnDex - 1 )
            sendDex  = randrange(0,Ntracks-1)
            try:
                response = spot.playlist_reorder_items(
                    playlist_ID, 
                    bgnDex, 
                    sendDex, 
                    range_length = chunkSize
                )
                if verbose:
                    print( f"Iteration {i+1}, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{response}" )
                fail = False
            except spotipy.SpotifyException as ex:
                if verbose:
                    print( f"ERROR, {bgnDex}-{bgnDex+moveSize} --> {sendDex}:\n\t{ex}" )


def complete_reorder_entire_playlist( playlist_ID, verbose = False, phasePause_s = 5 ):
    """ Completely shuffle the playlist in decreasing chunks """
    Ntracks = get_playlist_length( playlist_ID )
    ## Phase 1: Chunks of 100 ##
    if verbose:
        print("\nPhase 1: Chunks of 100\n")
    reorder_playlist_by_chunks( playlist_ID, 100, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 2: Chunks of 10 ##
    if verbose:
        print("\nPhase 2: Chunks of 10\n")
    reorder_playlist_by_chunks( playlist_ID, 10, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: Chunks of 5, 1/2 ##
    if verbose:
        print("\nPhase 3: Chunks of 5, 1/2\n")
    Nmoves = int( ceil( Ntracks / 10 ) )
    reorder_playlist_by_chunks( playlist_ID, 5, Nmoves, verbose = verbose )
    sleep( phasePause_s )
    ## Phase 3: 1 at a Time, 1/3 ##
    if verbose:
        print("\nPhase 3: 1 at a Time, 1/3\n")
    Nmoves = int( ceil( Ntracks / 3 ) )
    reorder_playlist_by_chunks( playlist_ID, 1, Nmoves, verbose = verbose )

# Shuffle Playlists!

In [3]:
playlist = {
    'study01' : "0a2qoe6S7lYeZ6nlhZdA0v",
    'study02' : "6gbtR2cBq5PvkghidCvvGk",
    'study03' : "3o3lN2qntdEV7UKTuuC77K",
    'study04' : "41sFSisljvBDMBXtpp5NIw",
    'study05' : "02iS5AFGp8YVuUUqcQf8ys",
    'study06' : "6KI7A4MWrSM7EyKRUjxIi1",
    'study07' : "3V055Md2JdrUT8tX0af7di",
    'study08' : "0tspdJlwSgiyf2O9PO6QaP",
    'study09' : "5mHRBFoQtYy2izeZ66pG95",
    'study10' : "3832xeKGEOAXFJqE4K8kIq",
    'study11' : "65MXR4dubPL9t0P4dgTWvn",
    'study12' : "0ecSAfnD4CulIVnLt26ukI",
    'study13' : "7K9ucByFRgDuZk8KMHeJkL",
    'study14' : "0v26bHydUxcGC5EbMlkjzG",
    'study15' : "6SqlfurCBP7eeMOojaDNtS",
    'study16' : "5TtKaKCouyJp7Hhtu4YlYm",
    'study17' : "5qX1Tq3IQ74iSgrlMfRhty",
    'study18' : "1bwXMYoRgEFWebWZ9ZPBqs",
    'study19' : "3ackEOD2vox5Oc1vjeALKJ", 
    'study20' : "3Cmpe4nGzOQDefowW49pBS",
    'PianoJz' : "2IqcxvGty13ewL583z9Kdn",
    'cringe1' : "2AAUYlKM1nXKHkZUpUSFbv",
}

complete_reorder_entire_playlist( playlist['study12'], verbose = True, phasePause_s = 5 )


Phase 1: Chunks of 100



HTTP Error for PUT to https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


ERROR, 400-405 --> 309:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks:
 Tracks selected to be reordered are out of bounds, reason: None


HTTP Error for PUT to https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 2, 200-300 --> 29:
	{'snapshot_id': 'AAAFDTlQFh8jTnDGdFTHD+5A7aWosCaX'}
ERROR, 400-405 --> 96:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 4, 300-400 --> 73:
	{'snapshot_id': 'AAAFDvQ0bm5hAzia92ey1qN/0G+yPbBG'}

Phase 2: Chunks of 10

Iteration 1, 350-360 --> 33:
	{'snapshot_id': 'AAAFD+zqMao6MFN2bNWszJ5aE2Mx6wBW'}
Iteration 2, 350-360 --> 340:
	{'snapshot_id': 'AAAFEPaWqt1bg84G8gukZzp3S99OH67x'}
Iteration 3, 160-170 --> 104:
	{'snapshot_id': 'AAAFEWTRQbG7ccvg/4Fa4xNEseN1cfxK'}
Iteration 4, 200-210 --> 276:
	{'snapshot_id': 'AAAFEoH7MiR8PSFoSL0eAkC53VPN7Pc6'}
Iteration 5, 300-310 --> 394:
	{'snapshot_id': 'AAAFEyUlrpgSL3n9QVjFiNOT66LfZPLg'}
Iteration 6, 80-90 --> 302:
	{'snapshot_id': 'AAAFFM8+fOBuSDu3OhaM/F8XY4y6Py3d'}
Iteration 7, 200-210 --> 41:
	{'snapshot_id': 'AAAFFVrJoaelkCf5CPRhEUQrBqDl49RX'}
Iteration 8, 0-10 --> 300:
	{'snapshot_id': 'AAAF

HTTP Error for PUT to https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks with Params: {} returned 400 due to Tracks selected to be reordered are out of bounds


Iteration 4, 240-245 --> 275:
	{'snapshot_id': 'AAAFOd1J4AiTvhKZ6xosQ6yPbDSqsfYt'}
ERROR, 405-405 --> 404:
	http status: 400, code: -1 - https://api.spotify.com/v1/playlists/0ecSAfnD4CulIVnLt26ukI/tracks:
 Tracks selected to be reordered are out of bounds, reason: None
Iteration 6, 255-260 --> 228:
	{'snapshot_id': 'AAAFOv/ulRENLshj3MHO+e/MX3AfIwzi'}
Iteration 7, 100-105 --> 52:
	{'snapshot_id': 'AAAFO6fmpWfbmAZUA0nxbXStfG8pVeoP'}
Iteration 8, 210-215 --> 302:
	{'snapshot_id': 'AAAFPPP/7ES6B1INoCMBq2Db7EisKFO3'}
Iteration 9, 305-310 --> 4:
	{'snapshot_id': 'AAAFPbHzx9XG9PBHip2oQibvQhX5r+rS'}
Iteration 10, 10-15 --> 168:
	{'snapshot_id': 'AAAFPm4H8rcXrVvVyL7b3ucgZt/1vVch'}
Iteration 11, 75-80 --> 331:
	{'snapshot_id': 'AAAFP+bQdL3NvqBruUVaUYIvfOyxICcp'}
Iteration 12, 50-55 --> 267:
	{'snapshot_id': 'AAAFQC6X6t/MPpv7q4jS4v7S8+hCNO9k'}
Iteration 13, 30-35 --> 127:
	{'snapshot_id': 'AAAFQQAzUrW0JB9GSDbVJChetc5GAMun'}
Iteration 14, 95-100 --> 36:
	{'snapshot_id': 'AAAFQhbxdOW8pK9MEy9dRyyHut